In [89]:
import pandas as pd

In [90]:
df = pd.read_csv('rawdata.csv')

In [91]:
len(df.index)

33

We want to make sure there are no duplicate responses in the survey. Since 'Submission ID' is always unique, we generate a list of columns in the data frame excluding 'Submission ID'

In [92]:
dup_cols = [col for col in df.columns if col not in ['Submission ID', 'Submitted at']]

In [93]:
df_clean = df.drop_duplicates(subset=dup_cols)

In [94]:
len(df_clean)

32

We have removed one duplicate response which had the same answers and the same 'Respondent ID'. That leaves 32 responses. Next, we want to see how many responses we have for each PGY level. 

In [95]:
df_response = df_clean.groupby('PGY year?').agg(
    count = ('Submission ID', 'nunique')
)

df_response

,count
PGY year?,
PGY1,7
PGY2,5
PGY3,5
PGY4,4
PGY5,4
Research Resident,7


We will now calculate the number of respondents who said they were not eligible to vote in the 2022 U.S. Midterm Election.

In [96]:
(df_clean['Did you vote in the 2022 Midterm Election?'] == 'I am not eligible to vote (i.e. non-US citizen)').sum()

4

In [97]:
df_eligible = df_clean[df_clean['Did you vote in the 2022 Midterm Election?'] != 'I am not eligible to vote (i.e. non-US citizen)']

In [98]:
len(df_eligible.index)

28

We have 28 respondents who were eligible to vote in the election. Next, we will create a new column indicating whether each eligible respondent voted (1) or did not vote (0).

In [99]:
def did_vote(response):
    if response == 'Yes':
        return 1
    else:
        return 0

df_eligible['did_vote'] = df_eligible.apply(
    lambda x:
    did_vote(x['Did you vote in the 2022 Midterm Election?']),
    axis=1
)


/var/folders/tc/rrbf67xd4h98qrpph8v1h9c00000gn/T/ipykernel_21046/815959324.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_eligible['did_vote'] = df_eligible.apply(


Now we will look at where respondents are registered to vote. 14 are registered to vote in the same state as our institution (New York) and two are unsure.

In [100]:
df_samestate = df_eligible.groupby('Are you registered to vote in the same state as your institution?').agg(
    count = ('Respondent ID', 'nunique')
)

df_samestate

,count
Are you registered to vote in the same state as your institution?,
I am not sure,2
No,12
Yes,14


Among those who are not registered in New York or unsure, we will see where they are registered.

In [101]:
df_state = df_eligible.groupby('In which state are you registered?').agg(
    count = ('Respondent ID', 'nunique')
)

df_state

,count
In which state are you registered?,
Alabama (AL),1
California (CA),1
Florida (FL),1
New Jersey (NJ),5
Ohio (OH),1
Pennsylvania (PA),2
Virginia (VA),1


We will now calculate the sum of the new column, which will give us the number of respondents who voted.

In [102]:
df_eligible['did_vote'].sum()

13

Dividing that number by the total respondents yields overall turnout

In [103]:
turnout = df_eligible['did_vote'].sum()/len(df_eligible.index)
turnout

0.4642857142857143

Now we will look at turnout by PGY Level

In [104]:
df_turnout = df_eligible.groupby('PGY year?').agg(
    vote = ('did_vote', 'sum'),
    count = ('Respondent ID', 'nunique')
)

df_turnout.reset_index(inplace=True)
df_turnout['pct_voting'] = df_turnout['vote']/df_turnout['count']
df_turnout

,PGY year?,vote,count,pct_voting
0,PGY1,3,7,0.428571
1,PGY2,1,4,0.250000
2,PGY3,3,5,0.600000
3,PGY4,1,4,0.250000
4,PGY5,0,3,0.000000
5,Research Resident,5,5,1.000000


Next we will look at barriers to voting amongst the respondents who did not vote. It looks like there are 15 non-voters.

In [105]:
df_not_vote = df_eligible[df_eligible['did_vote'] != 1]
len(df_not_vote.index)

15

We need to get rid of some of the excess text around the columns corresponding to barrier options.

In [106]:
df_not_vote.columns = df_not_vote.columns.str.replace(r'What reasons contributed to your decision not to vote\?', '')
df_not_vote.columns = df_not_vote.columns.str.replace(r'\n\(Select all that apply\)', '')
df_not_vote.columns = df_not_vote.columns.str.replace(r'\(', '')
df_not_vote.columns = df_not_vote.columns.str.replace(r'\)', '')

/var/folders/tc/rrbf67xd4h98qrpph8v1h9c00000gn/T/ipykernel_21046/1806834040.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_not_vote.columns = df_not_vote.columns.str.replace(r'What reasons contributed to your decision not to vote\?', '')
/var/folders/tc/rrbf67xd4h98qrpph8v1h9c00000gn/T/ipykernel_21046/1806834040.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_not_vote.columns = df_not_vote.columns.str.replace(r'\n\(Select all that apply\)', '')
/var/folders/tc/rrbf67xd4h98qrpph8v1h9c00000gn/T/ipykernel_21046/1806834040.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df_not_vote.columns = df_not_vote.columns.str.replace(r'\(', '')
/var/folders/tc/rrbf67xd4h98qrpph8v1h9c00000gn/T/ipykernel_21046/1806834040.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df_not_vote.col

We also want to identify the columns corresponding to the specific barriers

In [107]:
barrier_cols = df_not_vote.columns[13:22].to_list()
barrier_cols

[' I am unsure how to register to vote, if I am registered, or where I am registered',
 ' I am not sure how to request a mail-in ballot or what the deadlines for submission are',
 " I don't know about the candidates or issues where I am registered",
 " I don't care about the outcome of the election or hold strong political beliefs",
 " I don't want to vote",
 ' I want to vote, but I am too busy ',
 ' There is no way to leave work for long enough time to vote',
 ' Other',
 'Please describe...']

In [108]:
df_melt = df_not_vote.melt(
    id_vars = ['Respondent ID'],
    value_vars = barrier_cols,
    var_name='barrier'
)

df_melt

,Respondent ID,barrier,value
0,0QaGly,"I am unsure how to register to vote, if I am ...",False
1,2E7VGA,"I am unsure how to register to vote, if I am ...",False
2,obvgJM,"I am unsure how to register to vote, if I am ...",False
3,1A6O84,"I am unsure how to register to vote, if I am ...",True
4,ODEzk7,"I am unsure how to register to vote, if I am ...",False
...,...,...,...
130,2EROze,Please describe...,NaN
131,PdP8QB,Please describe...,NaN
132,jaE5Q1,Please describe...,NaN
133,eqoKkk,Please describe...,NaN


In [109]:
df_barriers = df_melt.groupby('barrier').agg(
    count = ('value', 'sum'),
    total = ('Respondent ID', 'nunique')
)

df_barriers.reset_index(inplace=True)
df_barriers

,barrier,count,total
0,I am not sure how to request a mail-in ballot...,6,15
1,"I am unsure how to register to vote, if I am ...",2,15
2,I don't care about the outcome of the electio...,1,15
3,I don't know about the candidates or issues w...,3,15
4,I don't want to vote,0,15
5,"I want to vote, but I am too busy",9,15
6,Other,1,15
7,There is no way to leave work for long enough...,7,15
8,Please describe...,Missed absentee ballot deadline,15


There was one respondent who filled out the other column and wrote that they missed the absentee ballot deadline. Given that absentee voting is related to mail in voting, we will add that to the mail in ballot count to make it 7.

In [110]:
df_barriers['count'] = df_barriers['count'].replace(6, 7)
df_barriers = df_barriers[~df_barriers['barrier'].isin(['Other', 'Please describe...'])]
df_barriers.sort_values(by='count', ascending=False, inplace=True)
df_barriers['pct'] = df_barriers['count']/df_barriers['total']
df_barriers

/var/folders/tc/rrbf67xd4h98qrpph8v1h9c00000gn/T/ipykernel_21046/338483807.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_barriers.sort_values(by='count', ascending=False, inplace=True)
/var/folders/tc/rrbf67xd4h98qrpph8v1h9c00000gn/T/ipykernel_21046/338483807.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_barriers['pct'] = df_barriers['count']/df_barriers['total']


,barrier,count,total,pct
5,"I want to vote, but I am too busy",9,15,0.6
0,I am not sure how to request a mail-in ballot...,7,15,0.466667
7,There is no way to leave work for long enough...,7,15,0.466667
3,I don't know about the candidates or issues w...,3,15,0.2
1,"I am unsure how to register to vote, if I am ...",2,15,0.133333
2,I don't care about the outcome of the electio...,1,15,0.066667
6,Other,1,15,0.066667
4,I don't want to vote,0,15,0.0
